In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path().absolute().parent))
import time

In [2]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
import torch
from torch.nn import MSELoss
from torch.optim import Adam
import matplotlib.pyplot as plt
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import glob
from pathlib import Path

from src.data_models.camels_ch import CamelsCH, CamelsCHConfig, get_all_gauge_ids
from src.data_models.dataset import HydroDataset
from src.data_models.preprocessing import (
    scale_time_series,
    scale_static_attributes,
    inverse_scale_static_attributes,
)
from src.data_models.datamodule import HydroDataModule

---

# 1. Load and prepare CAMELS-CH data

In [3]:
camels_config = CamelsCHConfig(
    timeseries_dir="/Users/cooper/Desktop/CAMELS-CH/data/timeseries/observation_based/",
    timeseries_pattern="CAMELS_CH_obs_based_*.csv",
    static_attributes_dir="/Users/cooper/Desktop/CAMELS-CH/data/static_attributes",
    use_climate=False,
    use_geology=False,
    use_glacier=False,
    use_human_influence=False,
    use_hydrogeology=False,
    use_hydrology=False,
    use_landcover=False,
    use_soil=False,
    use_topographic=False,
)

In [4]:
all_gauge_ids = get_all_gauge_ids(camels_config)

ids_for_training = all_gauge_ids[:10]

camels = CamelsCH(camels_config)
camels.load_stations(ids_for_training)

Loaded time series data for 10 stations


In [5]:
# static = camels.get_static_attributes()
# # q_mean	runoff_ratio	stream_elas	slope_fdc	baseflow_index_landson	hfd_mean
# static = static[["gauge_id", "q_mean", "runoff_ratio", "stream_elas", "slope_fdc", "baseflow_index_landson", "hfd_mean"]]
# static

## Get time series data


In [6]:
data = camels.get_time_series()
data = data[
    [
        "gauge_id",
        "date",
        "discharge_spec(mm/d)",
        "precipitation(mm/d)",
        "temperature_mean(degC)",
    ]
]

# # Split into train/test
# df_train, df_test = train_validate_split(data, train_ratio=0.8)

# # Scale by basin
# scaled_train, scaled_test, params = scale_time_series(
#     df_train,
#     df_test,
#     features=["discharge_spec(mm/d)", "precipitation(mm/d)", "temperature_mean(degC)"],
#     by_basin=True,  # Set False for global scaling
# )

In [7]:
# attributes = [col for col in static.columns if col != 'gauge_id']

# # Scale the attributes
# scaled_static_df, scaling_params = scale_static_attributes(static, attributes)

# # Add back gauge_id if needed
# scaled_static_df['gauge_id'] = static['gauge_id']

# # Inverse scale to verify
# original_df = inverse_scale_static_attributes(scaled_static_df[attributes], scaling_params)
# original_df['gauge_id'] = static['gauge_id']

# # Print results
# print("\nOriginal values:")
# print(static)
# print("\nScaled values:")
# print(scaled_static_df)
# print("\nInverse scaled values:")
# print(original_df)

# # Verify the scaling worked correctly
# np.testing.assert_array_almost_equal(
#     static[attributes].values,
#     original_df[attributes].values
# )
# print("\nVerification passed: Original and inverse-scaled values match!")

# 2. Configure preprocessing

In [8]:
preprocessing_config = {
    "features": {
        "scale_method": "per_basin",
        "log_transform": []
    },
    "target": {
        "scale_method": "per_basin",
        "log_transform": False
    },
    "static_features": {
        "scale_method": "global"
    }
}

# 3. Create DataModule

In [9]:
data_module = HydroDataModule(
    time_series_df=data,
    static_df=None,
    preprocessing_config=preprocessing_config,
    batch_size=32,
    input_length=30,
    output_length=1,
    num_workers=4,
    features=["discharge_spec(mm/d)", "precipitation(mm/d)", "temperature_mean(degC)"],
    target="discharge_spec(mm/d)",
    train_years=20,
    val_years=1,
    min_test_years=1,
)

## Wrap the data in a PyTorch dataset

In [10]:
# features = ["discharge_spec(mm/d)", "precipitation(mm/d)", "temperature_mean(degC)"]
# target = "discharge_spec(mm/d)"

# # Create dataset
# train_dataset = HydroDataset(
#     time_series_df=scaled_train,
#     static_df=None,
#     input_length=30,
#     output_length=5,
#     features=features,
#     target=target,
# )

# validate_dataset = HydroDataset(
#     time_series_df=scaled_train,
#     static_df=None,
#     input_length=30,
#     output_length=5,
#     features=features,
#     target=target,
# )

## Make the dataset iterable by creating a DataLoader

In [11]:
# batch_size = 32

# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# validate_loader = DataLoader(validate_dataset, batch_size=batch_size, shuffle=False)

## 4. Create model and train

In [12]:
from src.models.lstm import LitLSTM
from torch.optim import Adam
from torch.nn import MSELoss

In [13]:
model = LitLSTM(
    input_size=3,
    hidden_size=64,
    num_layers=1,
    output_size=1,
    target="discharge_spec(mm/d)",
)


# Configure trainer
trainer = pl.Trainer(
    max_epochs=5,
    accelerator="cpu",
    devices=1,
    callbacks=[
        ModelCheckpoint(
            monitor="val_loss",
            dirpath="checkpoints",
            filename="best-checkpoint",
            save_top_k=1,
            mode="min",
        ),
        EarlyStopping(monitor="val_loss", patience=3, mode="min"),
    ],
)

# Train the model
trainer.fit(model, data_module)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs



Quality Check Summary:
Original basins: 10
Retained basins: 4
Excluded basins: 6


/Users/cooper/Desktop/CAMELS-CH/.venv/lib/python3.12/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/Users/cooper/Desktop/CAMELS-CH/.venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
/Users/cooper/Desktop/CAMELS-CH/.venv/lib/python3.12/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /Users/cooper/Desktop/CAMELS-CH/notebooks/checkpoints exists and is not empty.

  | Name      | Type       | Params | M

Data preprocessing completed with temporal splitting:
- Using first 20 years of each basin for fitting scalers
- Features scaled using per_basin method
- Target scaled using per_basin method
- Log transforms applied to: [] and target: False
Created 29100 valid sequences.
Created 1340 valid sequences.

Data split summary:
Training: 29220 samples from 4 basins
Validation: 1460 samples from 4 basins
Testing: 27760 samples from 4 basins
Epoch 4: 100%|██████████| 910/910 [00:07<00:00, 127.68it/s, v_num=9]        

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 910/910 [00:07<00:00, 127.66it/s, v_num=9]


# 5. Evalue and plot results

In [14]:
quality_report = data_module.quality_report

excluded_basins = list(quality_report['excluded_basins'].keys())
excluded_basins

ids_for_training = [id for id in ids_for_training if id not in excluded_basins]

In [ ]:
trainer.test(model, data_module)
test_results = model.test_results

# Get predictions and transform
df_pred = pd.DataFrame(test_results["predictions"], columns=[model.hparams.target])
df_pred["gauge_id"] = test_results["basin_ids"]
df_targets = pd.DataFrame(test_results["targets"], columns=[model.hparams.target])
df_targets["gauge_id"] = test_results["basin_ids"]


Quality Check Summary:
Original basins: 10
Retained basins: 4
Excluded basins: 6
Data preprocessing completed with temporal splitting:
- Using first 20 years of each basin for fitting scalers
- Features scaled using per_basin method
- Target scaled using per_basin method
- Log transforms applied to: [] and target: False
Created 27640 valid sequences.

Data split summary:
Training: 29220 samples from 4 basins
Validation: 1460 samples from 4 basins
Testing: 27760 samples from 4 basins
Testing DataLoader 0: 100%|██████████| 864/864 [00:02<00:00, 343.06it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss          0.029699645936489105
────────────────────────────────────────────────────────────────────────────────────────────────────────

In [20]:
inversed_preds = data_module.inverse_transform_predictions(
    test_results["predictions"], basin_ids=test_results["basin_ids"]
).to_numpy()
inversed_targets = data_module.inverse_transform_predictions(
    test_results["targets"], basin_ids=test_results["basin_ids"]
).to_numpy()

inversed_preds

array([-0.11790432, -0.45984522, -0.5488223 , ...,  0.15254001,
       -0.04198272, -0.19649418], shape=(27640,), dtype=float32)

In [ ]:

# Plot for each basin
for specific_basin_id in ids_for_training:
    basin_indices = np.where(test_results["basin_ids"] == specific_basin_id)[0]

    plt.figure(figsize=(12, 6))
    sns.lineplot(
        x=range(min(365 * 3, len(inversed_targets[basin_indices]))),
        y=inversed_targets[basin_indices][: 365 * 3],
        label="Actual",
        color="blue",
    )
    sns.lineplot(
        x=range(min(365 * 3, len(inversed_preds[basin_indices]))),
        y=inversed_preds[basin_indices][: 365 * 3],
        label="Predicted",
        color="red",
        alpha=0.7,
    )

    plt.xlabel("Timestep", fontsize=12)
    plt.ylabel("Discharge", fontsize=12)
    plt.title(
        f"Predicted vs Actual Discharge for Basin {specific_basin_id}", fontsize=14
    )
    plt.legend(fontsize=12, loc="upper left")
    sns.despine()
    plt.show()